In [25]:
#Import libraries
import pandas as pd
!pip install pandas scikit-learn
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
#from sklearn.svm import OneClassSVM
from sklearn.metrics import confusion_matrix, classification_report

Defaulting to user installation because normal site-packages is not writeable


In [26]:
!pip install wordcloud
try:
    from wordcloud import WordCloud
    print("WordCloud imported successfully!")
except ModuleNotFoundError:
    print("WordCloud is not installed.")


Defaulting to user installation because normal site-packages is not writeable
WordCloud is not installed.


In [27]:
data = pd.read_csv('spam_ham_dataset.csv')
data

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,2933,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


In [28]:
#select relevant columns
data1 = data[['label','text']]

#removing subject improves model accuracy, as it reduces noise in the dataset
#data1['text'] = data1['text'].str[8
data1.loc[:, 'text'] = data1['text'].str[8:]
data1

,label,text
0,ham,enron methanol ; meter # : 988291\r\nthis is ...
1,ham,"hpl nom for january 9 , 2001\r\n( see attache..."
2,ham,"neon retreat\r\nho ho ho , we ' re around to ..."
3,spam,"photoshop , windows , office . cheap . main t..."
4,ham,re : indian springs\r\nthis deal is to book t...
...,...,...
5166,ham,put the 10 on the ft\r\nthe transport volumes...
5167,ham,3 / 4 / 2000 and following noms\r\nhpl can ' ...
5168,ham,calpine daily gas nomination\r\n>\r\n>\r\njul...
5169,ham,industrial worksheets for august 2000 activit...


In [29]:
#Visualize Word Distributions Using Word Clouds

#Separate spam and ham
spam = data1[data1['label'] == 'spam']
ham = data1[data1['label'] == 'ham']

In [30]:
'''
#Generate word clouds for spam and ham
def generate_wordcloud(text,title):
    wordcloud = WordCloud(width=800, height=800, background_color='white').generate(" ".join('text'))
    plt.figure(figsize=(10, 5))
    plt.imshow(worldcloud, interpolation='bilinear')
    plt.title(title)
    plt.axis('off')
    plt.show()

generate_wordcloud(spam['text'],'Spam Words')
generate_wordcloud(ham['text'],'Ham Words') 

'''

'\n#Generate word clouds for spam and ham\ndef generate_wordcloud(text,title):\n    wordcloud = WordCloud(width=800, height=800, background_color=\'white\').generate(" ".join(\'text\'))\n    plt.figure(figsize=(10, 5))\n    plt.imshow(worldcloud, interpolation=\'bilinear\')\n    plt.title(title)\n    plt.axis(\'off\')\n    plt.show()\n\ngenerate_wordcloud(spam[\'text\'],\'Spam Words\')\ngenerate_wordcloud(ham[\'text\'],\'Ham Words\') \n\n'

In [31]:
#Use CountVectorizer to create the DTM
vectorizer = CountVectorizer(lowercase=True, stop_words='english', min_df=5)
dtm = vectorizer.fit_transform(data1['text'])
dtm

<5171x8211 sparse matrix of type '<class 'numpy.int64'>'
	with 271275 stored elements in Compressed Sparse Row format>

In [32]:
#split dataset 80/20. with 80% used for training and 20% for testing
x_train, x_test, y_train, y_test = train_test_split(dtm, data1['label'], test_size=0.20, random_state=42)

In [33]:
#Convert counts to binary
x_train_bin = (x_train > 0).astype(int)
x_test_bin = (x_test > 0).astype(int)

In [34]:
#Train with Naive Bayes model
nb_spamfilter = MultinomialNB()
nb_spamfilter.fit(x_train_bin, y_train)

MultinomialNB()

In [35]:
#make predictions 
y_pred = nb_spamfilter.predict(x_test_bin)

# evaluate predictions
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=['ham','spam']))

[[710  32]
 [ 10 283]]
              precision    recall  f1-score   support

         ham       0.99      0.96      0.97       742
        spam       0.90      0.97      0.93       293

    accuracy                           0.96      1035
   macro avg       0.94      0.96      0.95      1035
weighted avg       0.96      0.96      0.96      1035

